In [11]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gemini"]
    }
)

llm_config = {
    "config_list" : config_list,
    "temperature" : 0.2,
    "timeout" : 300,
    "seed" : 4,
    "functions": [{
	"name": "translate",
	"description": "Translate Chinese into English and all non-Chinese into Traditional Chinese. All the words I send you need to be translated, and you only need to answer the translation results.",
	"parameters": {
		"type": "object",
		"properties": {
			"question": {
				"type": "string",
				"description": "The content would be translated.",
				}
			},
		"required": ["question"],
	},
	}]
}

In [6]:
import requests

API_URL = "http://127.0.0.1:3000/api/v1/prediction/097ae88c-eace-48f8-abdd-871e2dd07aa3"

def translate(question):
    response = requests.post(API_URL, json={"question": question})
    return response.json()

# 測試呼叫看看，可以正常呼叫
# output = translate({"question": "Hey, how are you?"})
# print(output["text"])

In [9]:
# 修改 system_message，以便任務達成後盡快結束對話。
assistant = autogen.AssistantAgent(
    name="assistant",
    # system_message="""You are a helpful assistant. Once the purpose is achieved, Add a new line with the content of "終止, TERMINATE" at the end.""",
    llm_config=llm_config
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",  # NEVER, TERMINATE, ALWAYS
    max_consecutive_auto_reply=3,
    # is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    # code_execution_config={"work_dir": "coding", "use_docker": False},
    llm_config = llm_config,
    # function_map={"translate": translate}
)

In [4]:
# 另外一種，註冊函數的方法 ????
user_proxy.register_function(
	function_map={
		"translate": translate
	}
)

In [12]:
context = input("請輸入要翻譯的內容: ")
user_proxy.initiate_chat(
    assistant, 
    message = "Run the 'translate' function to translate:" + context + ", and tell me a story about it in traditional chinese."
)

No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


user_proxy (to assistant):

Run the 'translate' function to translate:一坨大便, and tell me a story about it in traditional chinese.

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


APIStatusError: Error code: 405 - {'detail': 'Method Not Allowed'}